In [1]:
import torch
#from denoising_diffusion_pytorch import Unet, GaussianDiffusion
#from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D
from denoising_diffusion_pytorch import *
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import skew,kurtosis

In [ ]:
pip install torch torchvision numpy matplotlib einops ema_pytorch accelerate pytorch_fid

## Load the data

In [2]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [3]:
data = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [ ]:
# (40, 40, 99) (videonumber/trialnumber , channelnumber, datapoints)
data['sub01'].shape

In [ ]:
# matrix 1
data['sub01'][0].shape

In [ ]:
matrix = np.zeros((1280,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [ ]:
matrix.shape

In [ ]:
!mkdir images2
for i in range(1280):
    #transpose = np.transpose(matrix[i,:,:], (1,2,0))
    tra = np.multiply(matrix[i,:,:], 255).astype(np.uint8)
    img = Image.fromarray(tra)
    img.save("images2/train"+str(i)+".png")

----

DEPCREATED

In [ ]:
matrix = np.zeros((1280,3,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [4]:
matrix = np.zeros((1280,1,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [5]:
matrix.shape

(1280, 1, 40, 104)

In [6]:
np.isnan(matrix).any()

False

In [ ]:
matrix[0][0]

In [7]:
tensor_matrix = torch.tensor(matrix).float()

In [8]:
tensor_matrix.shape

torch.Size([1280, 1, 40, 104])

In [ ]:
image_size = (99,40)
image_size[0]

----

In [13]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

training_images = tensor_matrix # images are normalized from 0 to 1
#training_images = torch.rand(8, 3, 96, 40) # images are normalized from 0 to 1
loss = diffusion(training_images)
loss.backward()
# after a lot of training

sampled_images = diffusion.sample(batch_size = 4)
sampled_images.shape # (4, 3, 128, 128)

KeyboardInterrupt: 

In [12]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,           # number of steps
    sampling_timesteps = 250,   # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
    loss_type = 'l1'            # L1 or L2
)

trainer = Trainer(
    diffusion,
    'images2/',
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 500,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = True              # whether to calculate fid during training
)

trainer.train()

  0%|          | 0/700000 [00:00<?, ?it/s]

/opt/anaconda3/envs/BSc/lib/python3.10/site-packages/torch/autograd/__init__.py:197: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


NotImplementedError: The operator 'aten::native_group_norm_backward' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
for i in range(10):
    sampled_images = diffusion.sample(batch_size = 128)
    torch.save(sampled_images,'sampled1D_40-104_43_'+str(i))
    print("Finished Sample: ",i)